In [252]:
import glob
from lxml import etree
from bs4 import BeautifulSoup
import csv
import os.path
import json

In [2]:
holdings = glob.glob('holdings/*.html')

In [236]:
allrows  = []

for page in holdings:
    sn = page.replace('holdings/', '').replace('.html', '')
    with open(page, 'r') as fin:
        soup = BeautifulSoup(fin.read(), 'lxml')
        tree = etree.fromstring(soup.prettify())
    divstr = str(etree.tostring(tree.xpath('//div[@id = "std_box"]')[0])).replace('\\n', '\n')
    splits = divstr.split(str('HOLDING'))
    splits.pop(0)

    for record in splits:

        s = [l.strip() for l in record.split('\n')]

        location = s[0].replace(':','').strip()
        h4loc = s.index('<h4 class="holding">')
        holdingschunks = "\n".join(s[h4loc:]).split('Available')
        holdingschunks.pop(0)
        for line in holdingschunks:
            datechunks = line.split('\n')
            fmt = datechunks[0].replace("as:", '').strip()
            if 'Dates' in line:
                for i, l in enumerate(datechunks):
                    if "Dates:" in l:
                        start = i
            else:
                start = 0
            for i, l in enumerate(datechunks):
                if "Last updated:" in l:
                    stop = i
            datetext = BeautifulSoup("<f>" + "".join(datechunks[start + 1:stop]) + "</f>", 'lxml').get_text()
            allrows.append([sn, location, fmt, datetext])

In [237]:
allrows

[['2017234421',
  'California State Univ, San Marcos, San Marcos, CA',
  'Unspecified',
  ''],
 ['2017234421',
  'Univ of California, San Diego, La Jolla, CA',
  'Online Resource',
  ''],
 ['sn00062059',
  'Chicago Hist Soc Libr, Chicago, IL',
  'Original',
  '<2000:5:20>'],
 ['sn2001061344',
  'Univ of Illinois, Urbana, IL',
  'Unspecified',
  '1985:2:15,1986:1:24-2:7,4:4-25,7:25-8:7,22,1987:2:6,2:27-3:6,7:3-8:28,10:9-30,12:4,1988:1:8-15,1:29-2:19,3:17,31,4:14-5:19,6:2,9,23-12:29,1989:1:5-5:4,5:25-12:28,1990:1:4-3:15,29,6:14,28,10:4-12:27,1991:1:3-4:4'],
 ['sn2008020210',
  'Chicago Hist Soc Libr, Chicago, IL',
  'Microfilm Service Copy',
  '<1967:9:21>'],
 ['sn2008020210',
  'Univ of Illinois, Urbana, IL',
  'Unspecified',
  '1967:Sep.21'],
 ['sn2008060202',
  'Chicago Hist Soc Libr, Chicago, IL',
  'Microfilm Service Copy',
  '<1953:7:11>'],
 ['sn2008060202',
  'Univ of Illinois, Urbana, IL',
  'Unspecified',
  '1973:Jul.11'],
 ['sn2008060204',
  'Chicago Hist Soc Libr, Chicago, IL'

In [271]:
marcs = glob.glob('marc/*.xml')

# print(marcs)

morerows = []

for row in allrows:
    sn, location, fmt, datetext = row
#     sn = row[0]
    marc = 'marc/' + sn + '.xml'
    with open(marc, 'r') as fin:
        tree = etree.fromstring(fin.read())
    oclc = tree.xpath('//controlfield[@tag = "001"]/text()')[0]
    title = tree.xpath('//datafield[@tag = "245"]/subfield[@code = "a"]/text()')[0]
    freq = " ".join(tree.xpath('//datafield[@tag = "310"]//text()'))
    jsonname = 'jsons/' + sn + '.json'
    with open(jsonname, 'r') as fin:
        data = json.loads(fin.read())
    startyear = data['start_year']
    endyear = data['end_year']
    newrow = [sn, title, location, fmt, oclc, startyear, endyear, freq, datetext]
    morerows.append(newrow)

In [250]:
len(allrows)

402

In [272]:
with open('holdings.csv', 'w') as fout:
    csvout = csv.writer(fout)
    csvout.writerow(['sn', 'title', 'location', 'format', 'oclc', 'startyear', 'endyear', 'freq', 'material_dates'])
    csvout.writerows(morerows)

# Testing area

In [235]:
with open(page, 'r') as fin:
    soup = BeautifulSoup(fin.read(), 'lxml')
    tree = etree.fromstring(soup.prettify())

divstr = str(etree.tostring(tree.xpath('//div[@id = "std_box"]')[0])).replace('\\n', '\n')
splits = divstr.split(str('HOLDING'))
splits.pop(0)

for record in splits:
    
    s = [l.strip() for l in record.split('\n')]
    
    location = s[0].replace(':','').strip()
    h4loc = s.index('<h4 class="holding">')
    holdingschunks = "\n".join(s[h4loc:]).split('Available')
    holdingschunks.pop(0)
    for line in holdingschunks:
        datechunks = line.split('\n')
        fmt = datechunks[0].replace("as:", '').strip()
        if 'Dates' in line:
            for i, l in enumerate(datechunks):
                if "Dates:" in l:
                    start = i
        else:
            start = 0
        for i, l in enumerate(datechunks):
            if "Last updated:" in l:
                stop = i
        datetext = BeautifulSoup("<f>" + "".join(datechunks[start + 1:stop]) + "</f>", 'lxml').get_text()
        print(location, fmt, datetext)

Abraham Lincoln Presidential Libr, Springfield, IL Microfilm Service Copy <1984:1:12-6:28><1985:1:10-5:23, 7:25>
Abraham Lincoln Presidential Libr, Springfield, IL Microfilm Master <1984:1:12-6:28><1985:1:10-5:23, 7:25>
Chicago Hist Soc Libr, Chicago, IL Microfilm Service Copy <1984:5:24-6:28><1984:12:20-1985:2:7><1985:2:21-5:23>
Evanston Pub Libr, Evanston, IL Original <1984:1:12-1985:7:25>
Northwestern Univ, Evanston, IL Original <1985:5:16>
Univ of Illinois, Urbana, IL Microfilm Master <1984:5:24-6:28, 12:20-27><1985:1:3-2:7,21-5:23>
Univ of Illinois, Urbana, IL Microfilm Service Copy <1984:5:24-6:28, 12:20-27><1985:1:3-2:7,21-5:23>


In [149]:
divstr.xpath('//h4')

AttributeError: 'str' object has no attribute 'xpath'